In [8]:
# A* Algorithm using Manhattan distance

from queue import PriorityQueue

class PuzzleState:
    def __init__(self, tiles, zero_pos, g=0, parent=None):
        self.tiles = tiles  # Current state of the puzzle (1D list)
        self.zero_pos = zero_pos  # Position of the blank tile (0)
        self.g = g  # Cost to reach this state
        self.h = self.manhattan_distance()  # Heuristic cost
        self.f = self.g + self.h  # Total cost
        self.parent = parent  # To track the path

    def manhattan_distance(self):
        # Calculate the Manhattan distance for the current state
        goal_positions = {1: (0, 0), 2: (0, 1), 3: (0, 2),
                          8: (1, 0), 0: (1, 1), 4: (1, 2),
                          7: (2, 0), 6: (2, 1), 5: (2, 2)}
        distance = 0
        for i in range(9):
            if self.tiles[i] != 0:  # Skip the blank tile
                current_x, current_y = divmod(i, 3)
                goal_x, goal_y = goal_positions[self.tiles[i]]
                distance += abs(current_x - goal_x) + abs(current_y - goal_y)
        return distance

    def get_neighbors(self):
        # Generate valid moves from the current position of the blank tile
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
        zero_x, zero_y = self.zero_pos // 3, self.zero_pos % 3
       
        for dx, dy in directions:
            new_x, new_y = zero_x + dx, zero_y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_zero_pos = new_x * 3 + new_y
                new_tiles = self.tiles[:]
                # Swap the blank tile with the adjacent tile
                new_tiles[self.zero_pos], new_tiles[new_zero_pos] = new_tiles[new_zero_pos], new_tiles[self.zero_pos]
                neighbors.append(PuzzleState(new_tiles, new_zero_pos, self.g + 1, self))
       
        return neighbors

def a_star(initial_state):
    start_state = PuzzleState(initial_state, initial_state.index(0))
    open_list = PriorityQueue()
    closed_list = set()
   
    open_list.put((start_state.f, start_state))
   
    while not open_list.empty():
        _, current_state = open_list.get()

        if current_state.h == 0:  # If the heuristic is 0, we found the goal
            return reconstruct_path(current_state)
       
        closed_list.add(tuple(current_state.tiles))
       
        for neighbor in current_state.get_neighbors():
            if tuple(neighbor.tiles) in closed_list:
                continue
           
            if neighbor.f not in [item[0] for item in open_list.queue]:
                open_list.put((neighbor.f, neighbor))
   
    return None  # If no solution is found

def reconstruct_path(state):
    path = []
    while state:
        path.append(state.tiles)
        state = state.parent
    return path[::-1]  # Return the path from start to goal

def print_solution(solution):
    for step in solution:
        print(step)
        print("\n")

# Example usage
initial_state = [2, 8, 3,
                 1, 6, 4,
                 7, 0, 5]  # Example initial state
solution = a_star(initial_state)

if solution:
    print("Solution found:")
    print_solution(solution)
else:
    print("No solution found.")

Solution found:
[2, 8, 3, 1, 6, 4, 7, 0, 5]


[2, 8, 3, 1, 0, 4, 7, 6, 5]


[2, 0, 3, 1, 8, 4, 7, 6, 5]


[0, 2, 3, 1, 8, 4, 7, 6, 5]


[1, 2, 3, 0, 8, 4, 7, 6, 5]


[1, 2, 3, 8, 0, 4, 7, 6, 5]


